# Azure OpenAI Assistants using the Python OpenAI SDK

In this demo, we'll use Azure OpenAI assistants (preview) through the Python OpenAI SDK to automatically generate a TPS report for a fictional company called Contoso. We'll create an AI assistant to help us produce a plot from our data, gather insights from the plot, and then summarize everything in a report with coversheet included!

## Assistants concepts

Azure OpenAI Assistants allows you to create AI assistants tailored to your needs through custom instructions and augmented by advanced tools like code interpreter, and custom functions.

- **Assistants**: encapsulates the model, its instructions, tools, and (context) documents
- **Threads**: represents the state of a conversation
- **Runs**: executes an Assistant on a Thread, including the textual responses and multi-step tool use

## Code

Install the packages

In [ ]:
! pip install openai azure-identity pandas

### Configure the client

We'll configure the AzureOpenAI client and authenticate using Microsoft Entra ID (formerly Azure Active Directory).


In [9]:
import os
import dotenv
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

dotenv.load_dotenv()

client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
    api_version="2024-03-01-preview",
)

### 1. Create a plot from our data

We have csv data that contains an average latency for each resource over a 24 hour period. We'll start by reading the file with pandas and then creating a file so that assistants can use it with future requests. 

#### Upload the data

In [10]:
import pandas as pd

latency_data = pd.read_csv("data/contoso_data.csv")
latency_data.head(3)

,Date,Time,Resource A,Resource B,Resource C
0,3/25/2024,0:00,76,150,123
1,3/25/2024,1:00,349,209,115
2,3/25/2024,2:00,410,314,110


In [11]:
file = client.files.create(
  file=open('data/contoso_data.csv',"rb"),
  purpose='assistants',
)

#### Create an assistant

We'll start by creating an Assistant. This call will capture the instructions for the assistant, which tools are available to use, and model to use. In our case, we'll equip our assistant with the code interpreter tool which allows the Assistants API to write and run Python code in a sandboxed execution environment.

In [12]:
assistant = client.beta.assistants.create(
    instructions="You are a data scientist assistant. When given data and a query, \
    write the necessary code to create the proper visualization.",
    model="gpt-4-1106-preview",
    tools=[{"type": "code_interpreter"}],
)


#### Compose message and run thread

Next, we'll create a thread and add our message context for the assistant. We'll include the data file ID in the message since the assistant will need the file to complete the task. Now we can run the thread and wait for a response.

In [13]:
thread = client.beta.threads.create(
    messages=[
        {
        "role": "user",
        "content": "Visualize Latency over Time as a line plot across Resources, where the colors of the lines are red (Resource A), green (Resource B), and blue (Resource C). Title the plot 'Avg Latency (last 24 hrs)'. The plot should fit on single sheet of paper.",
        "file_ids": [file.id]
        }
    ]
)


In [14]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

We'll add a helper function, `wait_until_done` to help us ping the service for updates on the thread until it's done. We can see the "thought process" of the assistant with each update as it's working on our request.

In [26]:
import time

def wait_until_done(run):
    status = ""
    while status not in ["completed", "cancelled", "expired", "failed"]:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        status = run.status

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0].content[0]
        print(f"Assistant update: {latest_message.text.value}")
        time.sleep(10)

    if latest_message.type == "image_file":
        print("Image has been created!")
        return latest_message
    elif latest_message.type == "text" and \
        latest_message.text.annotations and \
            latest_message.text.annotations[0].file_path.file_id:
        print("File has been created!")
        return latest_message

    print(f"Something went wrong: {latest_message}")

In [16]:
message = wait_until_done(run)

Assistant update: Visualize Latency over Time as a line plot across Resources, where the colors of the lines are red (Resource A), green (Resource B), and blue (Resource C). Title the plot 'Avg Latency (last 24 hrs)'. The plot should fit on single sheet of paper.
Image has been created!


#### View the plot

Now we'll view the plot created and create a file for the assistant to use later.

In [17]:
plot_file_id = message.image_file.file_id
line_plot_path = "./images/line_plot3.png"
data = client.files.content(plot_file_id)
with open(line_plot_path, "wb") as file:
    file.write(data.read())

plot_file = client.files.create(
  file=open(line_plot_path, "rb"),
  purpose='assistants'
)

![line_plot](images/line_plot.png)

### 2. Get insights from our line plot

Since we created a data scientist assistant, let's ask it to provide insights from the plot we created in bullet point form so we can add it to our report.

In [18]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Provide the most important insights from the plot you just created in bullet point form. Call out any anomalies or trends you see."

)

client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

Run(id='run_amLSh3IwrN7wCTyvSb0k6EOe', assistant_id='asst_DJuwVZh8InhiuUn77gwyHPC6', cancelled_at=None, completed_at=None, created_at=1711751454, expires_at=1711752054, failed_at=None, file_ids=[], instructions='You are a data scientist assistant. When given data and a query,     write the necessary code to create the proper visualization.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_E76FwIRZ41HdN5qgpDJOVyRg', tools=[ToolAssistantToolsCode(type='code_interpreter')], usage=None)

### 3. Generate a cover sheet for our TPS Report

Because all TPS reports require a coversheet, let's use AI to do that for us! Here we'll use DALL-E to generate an image for our coversheet and then create the file so the assistant can use it later.

In [19]:
import httpx

response = client.images.generate(
    model='dall-e-3',
    prompt="Create an image that encompasses the term 'latency'",
)
cover_sheet_url = response.data[0].url
response = httpx.get(cover_sheet_url)

cover_sheet_path = './images/cover_sheet.png'


with open(cover_sheet_path, 'wb') as file:
  file.write(response.content)


cover_sheet = client.files.create(
  file=open(cover_sheet_path, "rb"),
  purpose='assistants'
)


![coversheet.png](images/cover_sheet.png)

### 4. Combine our plot, insights, and coversheet to a PDF

Finally, let's combine our plot, insights, and coversheet into a PDF file for our TPS report. We'll give the assistant explicit instructions on how to create the report.

In [25]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a PDF file TPS report for the company Contoso. IMPORTANT: \n\nFIRST PAGE: include title 'Contoso TPS Report' centered at the top. Underneath text, include the image file included in this message. Crop or reduce size of image to make sure it fits on the page.\n\nSECOND PAGE: include the plot image included in this message. Underneath the plot, include the bullet point insights that were generated from the data. Both the plot and insights MUST fit on the page, reducing size, or wrapping any text to the next line, if necessary. Output these TWO PAGES as a .pdf file. Make sure the output is two pages, with each page matching the instructions from this message.",
    file_ids=[cover_sheet.id, plot_file.id]
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


In [28]:
message = wait_until_done(run)

Assistant update: The TPS report for Contoso has been successfully created as a two-page PDF document. The first page includes the title and the first image, and the second page contains the plot image and the insights listed in bullet points. 

You can download the report from the following link:

[Download Contoso TPS Report PDF](sandbox:/mnt/data/Contoso_TPS_Report.pdf)
File has been created!


In [29]:
pdf_file_id = message.text.annotations[0].file_path.file_id
pdf_path = "./report/tps_report.pdf"

data = client.files.content(pdf_file_id) 
with open(pdf_path, "wb") as file:
    file.write(data.read())


And we're done! See [report/tps_report.pdf](report/tps_report.pdf) for the final report.